In [1]:
import os
from pyspark.find_spark_home import _find_spark_home
from pyspark.sql import SparkSession
from pyspark import StorageLevel, SparkConf, SparkContext
import sparknlp

print(_find_spark_home())

/home/zzfz/anaconda3/envs/BigData/lib/python3.10/site-packages/pyspark


In [2]:
# conf = (
#     SparkConf()
#     .setMaster("local[*]")
#     .set("spark.local.dir", "/home/zzfz/spark-temp")
#     .set("spark.driver.memory", "6g")
#     .set("spark.executor.memory", "12g")
#     .set("spark.driver.maxResultSize", "10g")
#     .set("spark.pyspark.python", "/home/zzfz/anaconda3/envs/BigData/bin/python")
#     .set("spark.pyspark.driver.python", "/home/zzfz/anaconda3/envs/BigData/bin/python")
# )

# spark = (
#     SparkSession.builder.appName("PySpark Hadoop Integration")
#     .config(conf=conf)
#     .getOrCreate()
# )

spark = sparknlp.start()

sc = spark.sparkContext

for item in sc.getConf().getAll():
    print(item)

your 131072x1 screen size is bogus. expect trouble
24/01/02 23:10:17 WARN Utils: Your hostname, GinXPS15 resolves to a loopback address: 127.0.1.1; using 172.29.117.66 instead (on interface eth0)
24/01/02 23:10:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/zzfz/anaconda3/envs/BigData/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/zzfz/.ivy2/cache
The jars for the packages stored in: /home/zzfz/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9197dbee-f2ee-452c-b0fc-ca5b6a6b319f;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.2.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failureacc

('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false')
('spark.driver.memory', '16G')
('spark.app.initial.file.urls', 'file:///home/zzfz/.ivy2/jars/javax.annotati

In [3]:
df = (
    spark.read.option("delimiter", "\t")
    .option("header", "true")
    .csv("hdfs://localhost:9000/datasets/Amazon_Reviews/*.tsv")
)

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [4]:
from pyspark.sql.functions import col, count, when

# Calculate the total number of rows
total_rows = df.count()

# Calculate the ratio of nulls for each column
null_rows = df.select([(count(when(col(c).isNull(), c))).alias(c) for c in df.columns])

# Show the null ratios
null_rows.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|            1753|       1787|         1794|       1794|1794|             1794|           2044|      12438|       8243|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [5]:
# Data Cleaning
df = df.na.drop()

In [6]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

# Convert columns to integer
df = df.withColumn("star_rating", col("star_rating").cast(IntegerType()))
df = df.withColumn("helpful_votes", col("helpful_votes").cast(IntegerType()))
df = df.withColumn("total_votes", col("total_votes").cast(IntegerType()))

# Convert 'Y'/'N' to True/False for verified_purchase
df = df.withColumn(
    "verified_purchase", when(col("verified_purchase") == "Y", True).otherwise(False)
)

# Convert 'Y'/'N' to True/False for vine
df = df.withColumn("vine", when(col("vine") == "Y", True).otherwise(False))

In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: boolean (nullable = false)
 |-- verified_purchase: boolean (nullable = false)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
# df.describe().show()

In [9]:
sampled_df = df.sample(fraction=0.00001, seed=3)

sampled_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+
|         US|   42925074|R23DAOA3M7YR6C|B00WAIUOEC|     613426562|Swim Sexy Women's...|         Apparel|          4|            0|          0|false|             true|          Four Stars|just what i thoug...| 2015-06-25|
|         US|   24053180|R3J26FX9RDO2WS|B00VIQMMK6|     125880002|Sakkas Desert Sun...|         Apparel|          4|

In [10]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, UniversalSentenceEncoder, SentimentDLModel
from pyspark.ml import Pipeline


def create_sentiment_pipeline(input_col):
    # Assemble Document
    documentAssembler = (
        DocumentAssembler().setInputCol(input_col).setOutputCol("document")
    )

    # Tokenize the document
    tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")

    # Use Universal Sentence Encoder for embeddings
    embeddings = (
        UniversalSentenceEncoder.pretrained()
        .setInputCols(["document"])
        .setOutputCol("sentence_embeddings")
    )

    # Load the pre-trained sentiment analysis model
    sentimentModel = (
        SentimentDLModel.pretrained("sentimentdl_use_twitter", "en")
        .setInputCols(["sentence_embeddings"])
        .setOutputCol("sentiment")
    )

    # Define the pipeline
    pipeline = Pipeline(
        stages=[documentAssembler, tokenizer, embeddings, sentimentModel]
    )

    return pipeline


# Create pipelines for review_headline and review_body
pipeline_headline = create_sentiment_pipeline("review_headline")
pipeline_body = create_sentiment_pipeline("review_body")

# # Fit and transform the pipelines
pipeline_model_headline = pipeline_headline.fit(sampled_df)
result_headline = pipeline_model_headline.transform(sampled_df)

pipeline_model_body = pipeline_body.fit(sampled_df)
result_body = pipeline_model_body.transform(sampled_df)

# Fit and transform the pipelines
# pipeline_model_headline = pipeline_headline.fit(df)
# result_headline = pipeline_model_headline.transform(df)

# pipeline_model_body = pipeline_body.fit(df)
# result_body = pipeline_model_body.transform(df)

# Display results
result_headline.select("review_headline", "sentiment.result").show(truncate=False)
result_body.select("review_body", "sentiment.result").show(truncate=False)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ — ]Download done! Loading the resource.
[ / ]

2024-01-02 23:11:50.359644: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ — ]

2024-01-02 23:11:53.027601: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2024-01-02 23:11:53.073115: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2024-01-02 23:11:53.107007: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2024-01-02 23:11:53.136769: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2024-01-02 23:11:53.167323: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.


[ \ ]

[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ | ]sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ — ]Download done! Loading the resource.
[OK!]
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]


24/01/02 23:12:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------------------------------------------------+----------+
|review_headline                                               |result    |
+--------------------------------------------------------------+----------+
|Four Stars                                                    |[positive]|
|Four Stars                                                    |[positive]|
|Five Stars                                                    |[positive]|
|but otherwise satisfied.                                      |[negative]|
|Not recommended to buy it is cheaply                          |[negative]|
|I ordered the 6X but the gown itself said size ...            |[negative]|
|Two Stars                                                     |[positive]|
|Fake product                                                  |[negative]|
|Five Stars                                                    |[positive]|
|disappointed                                                  |[negative]|
|Fits like a

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|review_body                                                                                                                                                                                                                                                                                                                                    

In [11]:
# Extract the 'result' column and rename it for clarity
result_headline = result_headline.select(
    "review_id", col("sentiment.result").alias("headline_sentiment")
)
result_body = result_body.select(
    "review_id", col("sentiment.result").alias("body_sentiment")
)

# Flatten the 'result' column as it's typically a list
result_headline = result_headline.withColumn(
    "headline_sentiment", col("headline_sentiment").getItem(0)
)
result_body = result_body.withColumn("body_sentiment", col("body_sentiment").getItem(0))

# Join the sentiment results back to the original DataFrame
sampled_df = sampled_df.join(result_headline, "review_id").join(
    result_body, "review_id"
)
# df = df.join(result_headline, "review_id").join(result_body, "review_id")

In [12]:
sampled_df.show()

+--------------+-----------+-----------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+------------------+--------------+
|     review_id|marketplace|customer_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|headline_sentiment|body_sentiment|
+--------------+-----------+-----------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+------------------+--------------+
|R23DAOA3M7YR6C|         US|   42925074|B00WAIUOEC|     613426562|Swim Sexy Women's...|         Apparel|          4|            0|          0|false|             true|          Four Stars|just what i thoug...| 2015-06-25|          posit

In [15]:
# Distribution of body_sentiment
body_sentiment_distribution = sampled_df.groupBy("body_sentiment").count()
body_sentiment_distribution.show()

# Distribution of headline_sentiment
headline_sentiment_distribution = sampled_df.groupBy("headline_sentiment").count()
headline_sentiment_distribution.show()

+--------------+-----+
|body_sentiment|count|
+--------------+-----+
|      positive|  662|
|      negative|  424|
|       neutral|   52|
+--------------+-----+



+------------------+-----+
|headline_sentiment|count|
+------------------+-----+
|          positive|  867|
|          negative|  246|
|           neutral|   25|
+------------------+-----+



In [19]:
from pyspark.sql import functions as F

# Group by product_category and body_sentiment, then count the occurrences
distribution = sampled_df.groupBy("product_category", "body_sentiment").count()

# Order the results for better readability (optional)
distribution = distribution.orderBy("product_category", "body_sentiment")

# Show the results
distribution.show(distribution.count(), truncate=False)

+----------------------+--------------+-----+
|product_category      |body_sentiment|count|
+----------------------+--------------+-----+
|Apparel               |negative      |24   |
|Apparel               |neutral       |4    |
|Apparel               |positive      |38   |
|Automotive            |negative      |20   |
|Automotive            |neutral       |1    |
|Automotive            |positive      |15   |
|Baby                  |negative      |7    |
|Baby                  |positive      |12   |
|Beauty                |negative      |28   |
|Beauty                |neutral       |4    |
|Beauty                |positive      |23   |
|Books                 |negative      |10   |
|Books                 |positive      |26   |
|Camera                |negative      |4    |
|Camera                |neutral       |2    |
|Camera                |positive      |14   |
|Digital_Ebook_Purchase|negative      |17   |
|Digital_Ebook_Purchase|neutral       |2    |
|Digital_Ebook_Purchase|positive  

In [20]:
from pyspark.sql import functions as F

# Group by product_category and body_sentiment, then count the occurrences
distribution = sampled_df.groupBy("product_category", "headline_sentiment").count()

# Order the results for better readability (optional)
distribution = distribution.orderBy("product_category", "headline_sentiment")

# Show the results
distribution.show(distribution.count(), truncate=False)

+----------------------+------------------+-----+
|product_category      |headline_sentiment|count|
+----------------------+------------------+-----+
|Apparel               |negative          |17   |
|Apparel               |positive          |49   |
|Automotive            |negative          |12   |
|Automotive            |positive          |24   |
|Baby                  |negative          |3    |
|Baby                  |neutral           |1    |
|Baby                  |positive          |15   |
|Beauty                |negative          |14   |
|Beauty                |positive          |41   |
|Books                 |negative          |11   |
|Books                 |neutral           |1    |
|Books                 |positive          |24   |
|Camera                |negative          |3    |
|Camera                |neutral           |2    |
|Camera                |positive          |15   |
|Digital_Ebook_Purchase|negative          |12   |
|Digital_Ebook_Purchase|neutral           |3    |


In [22]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType


def sentiment_to_number(sentiment):
    return {"positive": 1, "negative": -1, "neutral": 0}.get(sentiment, 0)


sentiment_udf = udf(sentiment_to_number, IntegerType())

# Apply the UDF to the sentiment columns
sampled_df = sampled_df.withColumn(
    "num_headline_sentiment", sentiment_udf("headline_sentiment")
)
sampled_df = sampled_df.withColumn(
    "num_body_sentiment", sentiment_udf("body_sentiment")
)
sampled_df.show()

# # Apply the UDF to the sentiment columns
# df = df.withColumn("num_headline_sentiment", sentiment_udf("headline_sentiment"))
# df = df.withColumn("num_body_sentiment", sentiment_udf("body_sentiment"))
# df.show()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
from pyspark.sql import functions as F


sampled_df = sampled_df.withColumn(
    "star_rating", sampled_df["star_rating"].cast("float")
)
sampled_df = sampled_df.withColumn(
    "num_headline_sentiment", sampled_df["num_headline_sentiment"].cast("float")
)
sampled_df = sampled_df.withColumn(
    "num_body_sentiment", sampled_df["num_body_sentiment"].cast("float")
)

category_stats = sampled_df.groupBy("product_category").agg(
    F.mean("star_rating").alias("mean_star_rating"),
    F.expr("percentile_approx(star_rating, 0.5)").alias(
        "median_star_rating"
    ),  # Approximate median
    F.mean("num_headline_sentiment").alias("mean_headline_sentiment"),
    F.mean("num_body_sentiment").alias("mean_body_sentiment"),
)


# df = df.withColumn("star_rating", df["star_rating"].cast("float"))
# df = df.withColumn("num_headline_sentiment", df["num_headline_sentiment"].cast("float"))
# df = df.withColumn("num_body_sentiment", df["num_body_sentiment"].cast("float"))

# category_stats = df.groupBy("product_category").agg(
#     F.mean("star_rating").alias("mean_star_rating"),
#     F.expr("percentile_approx(star_rating, 0.5)").alias(
#         "median_star_rating"
#     ),  # Approximate median
#     F.mean("num_headline_sentiment").alias("mean_headline_sentiment"),
#     F.mean("num_body_sentiment").alias("mean_body_sentiment"),
# )
category_stats.show(category_stats.count(), truncate=False)

+--------------------+------------------+------------------+-----------------------+--------------------+
|    product_category|  mean_star_rating|median_star_rating|mean_headline_sentiment| mean_body_sentiment|
+--------------------+------------------+------------------+-----------------------+--------------------+
|             Apparel| 3.893939393939394|               4.0|    0.48484848484848486| 0.21212121212121213|
|          Automotive| 4.055555555555555|               5.0|     0.3333333333333333| -0.1388888888888889|
|                Baby| 4.105263157894737|               5.0|      0.631578947368421|  0.2631578947368421|
|              Beauty|               4.0|               5.0|     0.4909090909090909|-0.09090909090909091|
|               Books|3.9166666666666665|               5.0|     0.3611111111111111|  0.4444444444444444|
|              Camera|               4.3|               5.0|                    0.6|                 0.5|
|Digital_Ebook_Pur...| 4.169491525423729|     

In [ ]:
from pyspark.sql.functions import col

# # Ensure the columns are cast to float for division
# df = df.withColumn("helpful_votes", col("helpful_votes").cast("float"))
# df = df.withColumn("total_votes", col("total_votes").cast("float"))

# # Calculate the ratio
# df = df.withColumn("helpful_ratio", col("helpful_votes") / col("total_votes"))

# Ensure the columns are cast to float for division
sampled_df = sampled_df.withColumn("helpful_votes", col("helpful_votes").cast("float"))
sampled_df = sampled_df.withColumn("total_votes", col("total_votes").cast("float"))

# Calculate the ratio
sampled_df = sampled_df.withColumn(
    "helpful_ratio", col("helpful_votes") / col("total_votes")
)

sampled_df.show()

In [ ]:
from pyspark.sql import functions as F

category_stats = sampled_df.groupBy("product_category").agg(
    F.mean("helpful_ratio").alias("mean_helpful_ratio"),
    F.expr("percentile_approx(helpful_ratio, 0.5)").alias("median_helpful_ratio"),
)

In [ ]:
sampled_df.cache()

In [ ]:
from pyspark.sql.functions import when

# Avoid division by zero and create a new column for the ratio
sampled_df = sampled_df.withColumn(
    "helpful_ratio",
    when(col("total_votes") != 0, col("helpful_votes") / col("total_votes")).otherwise(
        0
    ),
)

# Calculate correlation between star_rating and helpful_ratio
star_helpful_ratio_corr = sampled_df.stat.corr("star_rating", "helpful_ratio")
print(
    "Correlation between star_rating and helpful_votes/total_votes ratio:",
    star_helpful_ratio_corr,
)
# Calculate correlation between star_rating and helpful_votes
star_helpful_corr = sampled_df.stat.corr("star_rating", "helpful_votes")
print("Correlation between star_rating and helpful_votes:", star_helpful_corr)

# Calculate correlation between star_rating and total_votes
star_total_corr = sampled_df.stat.corr("star_rating", "total_votes")
print("Correlation between star_rating and total_votes:", star_total_corr)


# Avoid division by zero and create a new column for the ratio
sampled_df = sampled_df.withColumn(
    "helpful_ratio",
    when(col("total_votes") != 0, col("helpful_votes") / col("total_votes")).otherwise(
        0
    ),
)

# Calculate correlation between star_rating and helpful_ratio
star_helpful_ratio_corr = sampled_df.stat.corr("star_rating", "helpful_ratio")
print(
    "Correlation between star_rating and helpful_votes/total_votes ratio:",
    star_helpful_ratio_corr,
)

In [ ]:
stats_pd = category_stats.toPandas()

In [ ]:
import matplotlib.pyplot as plt


def plot_histogram(data, column, title, color):
    plt.figure(figsize=(10, 6))
    plt.hist(data[column], bins=20, color=color, edgecolor="black")
    plt.title(title)
    plt.xlabel(column)
    plt.ylabel("Frequency")
    plt.show()


plot_histogram(stats_pd, "mean_star_rating", "Mean Star Rating per Category", "skyblue")
plot_histogram(
    stats_pd, "median_star_rating", "Median Star Rating per Category", "olive"
)
plot_histogram(
    stats_pd, "mean_headline_sentiment", "Mean Headline Sentiment per Category", "gold"
)
plot_histogram(
    stats_pd, "mean_body_sentiment", "Mean Body Sentiment per Category", "teal"
)

In [ ]:
review_bodies = sampled_df.select("review_body").rdd.flatMap(lambda x: x).collect()
text = " ".join(review_bodies)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Generate a word cloud image
wordcloud = WordCloud(background_color="white").generate(text)

# Display the generated image:
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()